# Project Intro
【项目实现】： 任何完成项目的过程，都是由以下三步构成的。
<img src="pics/流程.png" width="40%">
这不是一个线性的过程，而可能出现“代码实现”碰壁后然后折返“分析过程”，再“代码实现”的情形。

【知识地图】：前面6关所讲的爬虫原理，在本质上，是一个我们所操作的对象在不断转换的过程。
<img src="pics/知识地图.png" width="40%">
从Response对象开始，我们就分成了两条路径，一条路径是数据放在HTML里，所以我们用BeautifulSoup库去解析数据和提取数据；另一条，数据作为Json存储起来，所以我们用response.json()方法去解析，然后提取、存储数据。



# 项目实操

## 明确目标

爬取知乎大v张佳玮的文章“标题”、“摘要”、“链接”，并存储到本地文件。

## 分析过程

URL：[https://www.zhihu.com/people/zhang-jia-wei/posts?page=1。](https://www.zhihu.com/people/zhang-jia-wei/posts?page=1%E3%80%82)
<img src="pics/分析1.png" width="40%">

点击右键——检查——Network，选All（而非XHR），然后刷新网页，点进去第0个请求:posts_by_votes，点Preview。
<img src="pics/分析2.png" width="40%">
发现有文章标题，看来数据是放在HTML里。走的应该是【知识地图】里上面那条路径。

去观察一下网页源代码了，我们点回Elements。
<img src="pics/分析3.png" width="40%">
发现文章标题对应的就是<a>元素里面的文本“大概，他们就是世上活得最明白的人吧。”还可以看到在<a>标签里面，还有属性target="_blank"，和data-za-detail-view-element_name="Title"。

按下command+f(windows电脑是ctrl+f)去搜索target。

<a标签>的上一个层级是<h2标签>，并且有class="ContentItem-title"。仍然用command+f搜索“ContentItem-title”，发现这个属性可以帮我们精准定位目标数据，可以用。

思路：

获取数据——用requests库；解析数据——用BeautifulSoup库；提取数据——用BeautifulSoup里的find_all()，翻页的话观察第一页，到最后一页的网址特征，写循环；存储数据——用csv和openpyxl都可以。基本思路就应该是这样。

## 代码实现

In [ ]:
import requests
from bs4 import BeautifulSoup
#引入requests和bs
url='https://www.zhihu.com/people/zhang-jia-wei/posts/posts_by_votes?page=1'
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
#使用headers是一种默认的习惯，默认你已经掌握啦~
res=requests.get(url,headers=headers)
#用resquest模块发起请求，将响应的结果赋值给变量res。
print(res.status_code)
#检查状态码

发现状态码显示200，表示请求成功，放心了，可以继续往下。

## 重新分析

打开Network，点开XHR，同时刷新页面，看到出现了很多个请求。
<img src="pics/分析4.png" width="40%">
看到两个带articles的请求，感觉有戏。点开首个articles看看preview，一层层点开，看到“title：记住就是一切”，猜测这是一个文章标题。
<img src="pics/分析5.png" width="40%">
在网页里面用command+f(windows电脑用ctrl+f)搜索一下“记住就是一切”，发现搜不到，奇怪。

那就看看跟首个articles请求长得很像的另一个articles的请求好啦，仍然看preview，看到title: "国产航母下水……让我想到李鸿章和北洋舰队"，仍然在网页里搜一下：
<img src="pics/分析6.png" width="40%">
观察第1页对第2页的请求，和第2页里对第3页请求的参数区别，是在headers里面的query string parameters里面。

发现除了offset都一样，offset代表起始值，limit表示加载的限制数，通过循环我们是可以爬到所有页数的内容了。

思路整理：
<img src="pics/思路.png" width="40%">
### 代码实现

In [ ]:
import requests
#引入requests
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
#封装headers
url='https://www.zhihu.com/api/v4/members/zhang-jia-wei/articles?'
#写入网址
params={
    'include':'data[*].comment_count,suggest_edit,is_normal,thumbnail_extra_info,thumbnail,can_comment,comment_permission,admin_closed_comment,content,voteup_count,created,updated,upvoted_followees,voting,review_info,is_labeled,label_info;data[*].author.badge[?(type=best_answerer)].topics',
    'offset':'10',
    'limit':'20',
    'sort_by':'voteups',
    }
#封装参数
res=requests.get(url,headers=headers,params=params)
#发送请求，并把响应内容赋值到变量res里面
print(res.status_code)
#确认请求成功

根据json数据结构来提取我们想要的文章标题数据。
<img src="pics/分析7.png" width="40%">

最外层是一个很大的字典，里面有两大元素，data:和paging:，这两大元素又是键值对应的字典形式，data这个键所对应的值是一个列表，里面有10元素，每个元素又是字典形式。

In [ ]:
import requests
#引入requests
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
#封装headers
url='https://www.zhihu.com/api/v4/members/zhang-jia-wei/articles?'
#写入网址
params={
    'include':'data[*].comment_count,suggest_edit,is_normal,thumbnail_extra_info,thumbnail,can_comment,comment_permission,admin_closed_comment,content,voteup_count,created,updated,upvoted_followees,voting,review_info,is_labeled,label_info;data[*].author.badge[?(type=best_answerer)].topics',
    'offset':'10',
    'limit':'20',
    'sort_by':'voteups',
    }
#封装参数
res=requests.get(url,headers=headers,params=params)
#发送请求，并把响应内容赋值到变量res里面
print(res.status_code)
#确认这个Response对象状态正确    
articles=res.json()
#用response.json()方法去解析数据，并赋值到变量articles上面，此时的articles是一个
print(articles)
#打印这个json文件
data=articles['data']
#取出键为data的值。
for i in data:
    print(i['title'])
    print(i['url'])
    print(i['excerpt'])
    #遍历列表，拿到的是列表里的每一个元素，这些元素都是字典，再通过键把值取出来

看看第一页和最后一页请求的参数区别：

对比一下，你会发第一页的is_end是显示false，最后一页的is_end是显示true，这个元素可以帮我们结束循环。

设置为爬两页数据就停止。

In [ ]:
import requests
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
url='https://www.zhihu.com/api/v4/members/zhang-jia-wei/articles?'
articlelist=[]
#建立一个空列表，以待写入数据
offset=0
#设置offset的起始值为0
while True:
    params={
        'include':'data[*].comment_count,suggest_edit,is_normal,thumbnail_extra_info,thumbnail,can_comment,comment_permission,admin_closed_comment,content,voteup_count,created,updated,upvoted_followees,voting,review_info,is_labeled,label_info;data[*].author.badge[?(type=best_answerer)].topics',
        'offset':str(offset),
        'limit':'20',
        'sort_by':'voteups',
        }
    #封装参数
    res=requests.get(url,headers=headers,params=params)
    #发送请求，并把响应内容赋值到变量res里面
    articles=res.json()
    # print(articles)
    data=articles['data']
    #定位数据
    for i in data:
        list1=[i['title'],i['url'],i['excerpt']]
        #把数据封装成列表
        articlelist.append(list1) 
    offset=offset+20
    #在while循环内部，offset的值每次增加20
    if offset>40:
        break
    #如果offset大于40，即爬了两页，就停止
    #if articles['paging']['is_end'] == True:
    #如果键is_end所对应的值是True，就结束while循环。
        #break
print(articlelist)
#打印看看

In [ ]:
# 接下来就是存储数据。

import requests
import csv
#引用csv。
csv_file=open('articles.csv','w',newline='',encoding='utf-8')
#调用open()函数打开csv文件，传入参数：文件名“articles.csv”、写入模式“w”、newline=''。
writer = csv.writer(csv_file)
# 用csv.writer()函数创建一个writer对象。
list2=['标题','链接','摘要']
#创建一个列表
writer.writerow(list2)
#调用writer对象的writerow()方法，可以在csv文件里写入一行文字 “标题”和“链接”和"摘要"。

headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
url='https://www.zhihu.com/api/v4/members/zhang-jia-wei/articles?'
offset=0
#设置offset的起始值为0
while True:
    params={
        'include':'data[*].comment_count,suggest_edit,is_normal,thumbnail_extra_info,thumbnail,can_comment,comment_permission,admin_closed_comment,content,voteup_count,created,updated,upvoted_followees,voting,review_info,is_labeled,label_info;data[*].author.badge[?(type=best_answerer)].topics',
        'offset':str(offset),
        'limit':'20',
        'sort_by':'voteups',
        }
    #封装参数
    res=requests.get(url,headers=headers,params=params)
    #发送请求，并把响应内容赋值到变量res里面
    articles=res.json()
    print(articles)
    data=articles['data']
    #定位数据
    for i in data:
        list1=[i['title'],i['url'],i['excerpt']]
        #把目标数据封装成一个列表
        writer.writerow(list1)
        #调用writerow()方法，把列表list1的内容写入
    offset=offset+20
    #在while循环内部，offset的值每次增加20
    if offset > 40:
        break
csv_file.close()
#写入完成后，关闭文件就大功告成
print('okay')

# 展望未来

第8关是运用cookies让浏览器记住我们，第9关是用selenium控制浏览器，第10关是让爬虫程序能定时向我们汇报结果，第11-14关是运用协程和scrapy框架来帮我们提速，并且可以爬取海量的数据，第15关是复习。